In [1]:
import wikipedia
import shapely
import urllib
import os


from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, BigInteger, String, Sequence, Float
from geoalchemy2 import Geometry

from sqlalchemy import create_engine

from sqlalchemy.orm import sessionmaker

from model.persons_datamodel import Person_handler, Person
from model.locations_datamodel import Location_handler, Location

from lib.wikipedia_read import TryFindWikiPage

image_folder = ""

In [2]:
lh = Location_handler()

In [3]:
Base = declarative_base()

class OSM_point(Base):
    __tablename__ = 'OSM_points'
    id = Column(Integer, Sequence('OSM_points_seq'), primary_key=True)
    geom = Column(Geometry('POINT'))
    full_id = Column(String(50))
    osm_id = Column(String(50))
    historic = Column(String(50))
    name = Column(String(50))
    tourism = Column(String(50))
    inscriptio = Column(String(50))
    material = Column(String(50))
    subject_wi = Column(String(50))
    wikidata = Column(String(50))
    start_date = Column(String(50))
    wikipedia = Column(String(50))
    name_en = Column(String(50))
    image = Column(String(50))
    descriptio = Column(String(50))
    height = Column(String(50))
    memorial = Column(String(50))
    date = Column(String(50))
    mapillary = Column(String(50))
    wikimedia_ = Column(String(50))

In [4]:
query = lh.session.query(OSM_point)#.filter(OSM_point.wikipedia != None)
print(query.count())

774


In [5]:
"""
class OSM_Wiki(Base):
    _max_summary_len = 5000
    __tablename__ = 'OSM_wiki'
    id = Column(BigInteger, Sequence('OSM_wiki_seq'), primary_key=True)
    osm_id = Column(BigInteger)
    osm_name = Column(String(100))
    wikiname = Column(String(100))
    summary = Column(String(_max_summary_len))
    geom = Column(Geometry('POINT'))
    wiki_geom = Column(Geometry('POINT'))
    wiki_img_url = Column(String(255))
    date = Column(String(50))
    

    
OSM_Wiki.__table__.drop(engine)
OSM_Wiki.__table__.create(engine) 
"""

"\nclass OSM_Wiki(Base):\n    _max_summary_len = 5000\n    __tablename__ = 'OSM_wiki'\n    id = Column(BigInteger, Sequence('OSM_wiki_seq'), primary_key=True)\n    osm_id = Column(BigInteger)\n    osm_name = Column(String(100))\n    wikiname = Column(String(100))\n    summary = Column(String(_max_summary_len))\n    geom = Column(Geometry('POINT'))\n    wiki_geom = Column(Geometry('POINT'))\n    wiki_img_url = Column(String(255))\n    date = Column(String(50))\n    \n\n    \nOSM_Wiki.__table__.drop(engine)\nOSM_Wiki.__table__.create(engine) \n"

In [6]:
#start looking for wiki pages
lang = 'dk'
i = 0

N = query.count()
for OSM_point in query:
    
    try_pages = [OSM_point.name]
    
    if OSM_point.name_en != None:
        try_pages.append(OSM_point.name_en)
        
    if OSM_point.wikipedia != None:
        page_title = OSM_point.wikipedia.split(':')[1]
        lang = OSM_point.wikipedia.split(':')[0]
        try_pages.insert(0, page_title)
    
    page = TryFindWikiPage(try_pages, lang, default_lang='dk')
    
    if page == None:
        continue
    
    i += 1
    
    max_summary_len = 5000
    
    summary = page.summary
    
    try:
        print(page.category)
    except:
        pass
    
    if len(summary) > max_summary_len:
        summary = summary[:max_summary_len-1]

    wiki_img_url = None
    #read image if exists
    if len(page.images) != 0:
        for url in page.images:

            ext = os.path.splitext(url)[1]
            if ext in ['.jpg', '.png', '.JPG', '.PNG']:
                img_file_name = "data/images/OSM_wiki/%s%s" %(page.title, ext)
                image=urllib.request.URLopener()
                image.retrieve(url,img_file_name)
                wiki_img_url = url
                break

    try:
        wiki_geom = "Point(%.6f %.6f)" %(page.coordinates[0],page.coordinates[1])
    except:
        wiki_geom = None

    location = Location(
        name = OSM_point.name,
        lang = lang,            #landekode
        name_en = OSM_point.name_en,
        geom = OSM_point.geom,
        historic = OSM_point.historic,
        category = OSM_point.tourism,
        description = OSM_point.descriptio,
        inscription = OSM_point.inscriptio,
        height = OSM_point.height,
        date_build = OSM_point.date,
        wiki_title = page.title,
        wiki_url = wiki_img_url,           #url
        wiki_page = summary,
        wiki_geom = wiki_geom,
        image_url = wiki_img_url,   #url 
        image_path = img_file_name, #url 
        person_relation = None,      #JSON key value pair med key= relation og value= id på personen + navn
        location_relation = None 
    )
    lh.UploadLocation(location)

    print(i)

1
2
3
4
5
6
7
8


/home/grusinator/anaconda3/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /home/grusinator/anaconda3/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
